# Carry-flag diagnostics with a matched control

This standalone notebook reproduces the carry-flag diagnostic study in the paper. It compares a mixed-layout coupled-significance base model, a matched extra-data control, and a targeted carry-probe repair.

Upload **this notebook only** to Google Colab, enable a GPU, and run all cells. No extra `.py` files or archives are required.

Compared families:

- **mixedlayout_pcsig_base**: base family after layout repair
- **mixedlayout_pcsig_ctrl_dup**: matched extra-data control
- **mixedlayout_pcsig_probe_all**: targeted carry-probe repair

Primary questions:

- Does the hundreds position behave like a carry flag rather than a semantic hundreds digit?
- Is the observed repair content-specific, rather than a by-product of extra optimization alone?


In [ ]:

from __future__ import annotations

# ====== Basic configuration ======
RUN_PROFILE = "carry_flag_diagnostics"
OUTPUT_ROOT = "results_carry_flag_diagnostics"
AUTO_DOWNLOAD_ZIP = True
FORCE_CLEAN_OUTPUT_ROOT = True
SAVE_MODEL_STATE = False

# Default hyperparameters
N_LAYER = 1
N_EMBD = 16
N_HEAD = 4
TARGET_EPOCH_EQUIV = 128.0
BATCH_SIZE = 256
LEARNING_RATE = 3e-3
WEIGHT_DECAY = 0.0
EVAL_N_PER_SUITE = 1000
BASE_SEEDS = [0, 1, 2, 3, 4]

print({
    "RUN_PROFILE": RUN_PROFILE,
    "OUTPUT_ROOT": OUTPUT_ROOT,
    "SAVE_MODEL_STATE": SAVE_MODEL_STATE,
    "N_LAYER": N_LAYER,
    "N_EMBD": N_EMBD,
    "N_HEAD": N_HEAD,
    "TARGET_EPOCH_EQUIV": TARGET_EPOCH_EQUIV,
    "BATCH_SIZE": BATCH_SIZE,
    "LEARNING_RATE": LEARNING_RATE,
    "WEIGHT_DECAY": WEIGHT_DECAY,
    "EVAL_N_PER_SUITE": EVAL_N_PER_SUITE,
    "BASE_SEEDS": BASE_SEEDS,
})

import json
import math
import os
import shutil
import zipfile
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

# ====== Embedded common module (standalone) ======
import base64
import sys
import types

_EMBEDDED_COMMON_B64 = """..."""

if "microgpt_tmlr_extension_common" not in sys.modules:
    _embedded_common = types.ModuleType("microgpt_tmlr_extension_common")
    exec(
        compile(
            base64.b64decode(_EMBEDDED_COMMON_B64).decode("utf-8"),
            "microgpt_tmlr_extension_common.py",
            "exec",
        ),
        _embedded_common.__dict__,
    )
    sys.modules["microgpt_tmlr_extension_common"] = _embedded_common

print("embedded module loaded: microgpt_tmlr_extension_common")

from microgpt_tmlr_extension_common import (
    PROBE_EXTRA_DOCS,
    build_eval_rows_from_pairs,
    build_full_train_docs_2digit,
    build_full_train_docs_3digit_lowrange,
    build_probe_h0_unique_docs,
    build_probe_h1_unique_docs,
    build_probe_t1_unique_docs,
    build_probe_pack,
    carry_vector_n_digits,
    format_addition_doc,
    required_block_size,
    sample_pairs_2digit,
    sample_pairs_true_3digit_conditioned,
    teacher_forced_position_logits,
    train_one_experiment,
)


device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)
if device == "cuda":
    print("gpu:", torch.cuda.get_device_name(0))


def build_design_manifest() -> pd.DataFrame:
    rows = [
        {
            "family": "mixedlayout_pcsig_base",
            "train_family": "mixedlayout",
            "model_kind": "coupled_sig",
            "extra_data_kind": "none",
            "analysis_label": "base",
        },
        {
            "family": "mixedlayout_pcsig_ctrl_dup",
            "train_family": "mixedlayout",
            "model_kind": "coupled_sig",
            "extra_data_kind": "ctrl_dup",
            "analysis_label": "matched_extra_data_control",
        },
        {
            "family": "mixedlayout_pcsig_probe_all",
            "train_family": "mixedlayout",
            "model_kind": "coupled_sig",
            "extra_data_kind": "probe_all",
            "analysis_label": "targeted_probe_all",
        },
    ]
    return pd.DataFrame(rows)


DESIGN_MANIFEST = build_design_manifest()
print(DESIGN_MANIFEST)


def build_train_docs(train_family: str, extra_data_kind: str):
    if train_family != "mixedlayout":
        raise ValueError(train_family)
    docs = build_full_train_docs_2digit() + build_full_train_docs_3digit_lowrange()
    docs = docs + build_probe_pack(extra_data_kind, total=PROBE_EXTRA_DOCS)
    return docs


def build_probe_eval_rows_from_docs(docs, suite_name):
    rows = []
    for doc in docs:
        prefix, target = doc.split("=")
        a_s, b_s = prefix.split("+")
        a = int(a_s)
        b = int(b_s)
        c1, c2, c3 = carry_vector_n_digits(a, b, 3)
        rows.append({
            "suite": suite_name,
            "a": a,
            "b": b,
            "doc": doc,
            "prefix": prefix + "=",
            "target": target,
            "y": a + b,
            "width_in": 3,
            "width_out": 4,
            "style": "zero_pad",
            "c1": c1,
            "c2": c2,
            "c3": c3,
        })
    return rows


def build_experiments():
    experiments = []
    block_size = max(required_block_size(2), required_block_size(3))  # 13
    for row in DESIGN_MANIFEST.to_dict(orient="records"):
        train_docs = build_train_docs(row["train_family"], row["extra_data_kind"])
        num_steps = int(math.ceil(float(TARGET_EPOCH_EQUIV) * len(train_docs) / max(1, BATCH_SIZE)))
        if num_steps % 250 != 0:
            num_steps = 250 * int(math.ceil(num_steps / 250))

        for seed in BASE_SEEDS:
            eval_rows = []
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_2digit(EVAL_N_PER_SUITE, seed=1000 + seed),
                width_in=2, width_out=3, suite="in_support_2digit",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_true_3digit_conditioned(
                    EVAL_N_PER_SUITE,
                    seed=3000 + seed,
                    incoming_carry_to_hundreds=0,
                    carryout_from_hundreds=0,
                ),
                width_in=3, width_out=4, suite="true3_noincarry",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_true_3digit_conditioned(
                    EVAL_N_PER_SUITE,
                    seed=4000 + seed,
                    incoming_carry_to_hundreds=1,
                    carryout_from_hundreds=0,
                ),
                width_in=3, width_out=4, suite="true3_withincarry",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_true_3digit_conditioned(
                    EVAL_N_PER_SUITE,
                    seed=5000 + seed,
                    carryout_from_hundreds=1,
                ),
                width_in=3, width_out=4, suite="true3_thousands_carry",
            ))
            experiments.append({
                "name": f"{row['family']}_seed{seed}",
                "family": row["family"],
                "train_family": row["train_family"],
                "model_kind": row["model_kind"],
                "extra_data_kind": row["extra_data_kind"],
                "analysis_label": row["analysis_label"],
                "seed": seed,
                "block_size": block_size,
                "num_steps": num_steps,
                "batch_size": BATCH_SIZE,
                "learning_rate": LEARNING_RATE,
                "weight_decay": WEIGHT_DECAY,
                "n_layer": N_LAYER,
                "n_embd": N_EMBD,
                "n_head": N_HEAD,
                "train_docs": train_docs,
                "eval_rows": eval_rows,
                "n_train_docs": len(train_docs),
                "n_eval": len(eval_rows),
            })
    return experiments


EXPERIMENTS = build_experiments()
print(pd.DataFrame([{
    "name": e["name"],
    "family": e["family"],
    "extra_data_kind": e["extra_data_kind"],
    "seed": e["seed"],
    "num_steps": e["num_steps"],
    "n_train_docs": e["n_train_docs"],
    "n_eval": e["n_eval"],
} for e in EXPERIMENTS]))


def analyze_hundreds_step(model, model_kind, stoi, itos, BOS, device, seed, family):
    probe_rows = []
    probe_rows.extend(build_probe_eval_rows_from_docs(build_probe_h0_unique_docs(), "probe_h0"))
    probe_rows.extend(build_probe_eval_rows_from_docs(build_probe_h1_unique_docs(), "probe_h1"))
    probe_rows.extend(build_probe_eval_rows_from_docs(build_probe_t1_unique_docs(), "probe_t1"))

    # canonical indices for width_in=3, current text='abc+def=g'
    # BOS=0, a_h=1, a_t=2, a_o=3, +=4, b_h=5, b_t=6, b_o=7, =8, y_th=9
    low2_positions = [2, 3, 6, 7]
    high_positions = [1, 5]
    prev_output_pos = [9]
    digit_ids = [stoi[str(i)] for i in range(10)]

    rows = []
    for row in probe_rows:
        target_prefix = row["target"][0]   # teacher-force thousands digit
        logits, hidden, att_maps = teacher_forced_position_logits(
            model_kind=model_kind,
            model=model,
            prefix=row["prefix"],
            target_prefix=target_prefix,
            stoi=stoi,
            BOS=BOS,
            device=device,
        )
        logits = logits[0]
        probs = torch.softmax(logits[digit_ids], dim=0)
        pred_digit = str(int(torch.argmax(logits[digit_ids]).item()))
        target_hundreds = row["target"][1]
        flag_digit = str(int(row["c2"]))

        logit_flag = float(logits[stoi[flag_digit]].item())
        logit_true = float(logits[stoi[target_hundreds]].item())
        prob_flag = float(probs[int(flag_digit)].item())
        prob_true = float(probs[int(target_hundreds)].item())
        hidden_norm = float(torch.norm(hidden[0]).item())

        final_att = att_maps[-1][0]   # [n_head, T, T]
        query_pos = final_att.shape[-2] - 1
        att_to_low2 = float(final_att[:, query_pos, low2_positions].sum(dim=-1).mean().item())
        att_to_high = float(final_att[:, query_pos, high_positions].sum(dim=-1).mean().item())
        att_to_prev_output = float(final_att[:, query_pos, prev_output_pos].sum(dim=-1).mean().item())

        rows.append({
            "family": family,
            "seed": seed,
            "suite": row["suite"],
            "a": row["a"],
            "b": row["b"],
            "c1": row["c1"],
            "c2": row["c2"],
            "c3": row["c3"],
            "target": row["target"],
            "target_thousands": row["target"][0],
            "target_hundreds": target_hundreds,
            "flag_digit": flag_digit,
            "pred_hundreds": pred_digit,
            "pred_hundreds_eq_c2": int(pred_digit == flag_digit),
            "pred_hundreds_eq_target": int(pred_digit == target_hundreds),
            "logit_flag": logit_flag,
            "logit_true": logit_true,
            "flag_minus_true": logit_flag - logit_true,
            "prob_flag": prob_flag,
            "prob_true": prob_true,
            "hidden_norm": hidden_norm,
            "att_to_low2": att_to_low2,
            "att_to_high": att_to_high,
            "att_to_prev_output": att_to_prev_output,
        })
    return pd.DataFrame(rows)


output_root = Path(OUTPUT_ROOT)
if FORCE_CLEAN_OUTPUT_ROOT and output_root.exists():
    shutil.rmtree(output_root)
output_root.mkdir(parents=True, exist_ok=True)

DESIGN_MANIFEST.to_csv(output_root / "design_manifest.csv", index=False)

suite_rows = []
activation_rows = []
manifest_rows = []

for exp in EXPERIMENTS:
    workdir = output_root / exp["name"]
    result = train_one_experiment(
        name=exp["name"],
        family=exp["family"],
        train_family=exp["train_family"],
        model_kind=exp["model_kind"],
        seed=exp["seed"],
        block_size=exp["block_size"],
        num_steps=exp["num_steps"],
        batch_size=exp["batch_size"],
        learning_rate=exp["learning_rate"],
        weight_decay=exp["weight_decay"],
        n_layer=exp["n_layer"],
        n_embd=exp["n_embd"],
        n_head=exp["n_head"],
        train_docs=exp["train_docs"],
        eval_rows=exp["eval_rows"],
        device=device,
        save_model_state=SAVE_MODEL_STATE,
        workdir=workdir,
    )
    suite_rows.append(result["suite_df"])

    activation_df = analyze_hundreds_step(
        model=result["model"],
        model_kind=exp["model_kind"],
        stoi=result["stoi"],
        itos=result["itos"],
        BOS=result["BOS"],
        device=device,
        seed=exp["seed"],
        family=exp["family"],
    )
    activation_df.to_csv(workdir / "carryflag_activation_raw.csv", index=False)
    activation_rows.append(activation_df)

    manifest_rows.append({
        "name": exp["name"],
        "family": exp["family"],
        "train_family": exp["train_family"],
        "model_kind": exp["model_kind"],
        "extra_data_kind": exp["extra_data_kind"],
        "analysis_label": exp["analysis_label"],
        "seed": exp["seed"],
        "block_size": exp["block_size"],
        "num_steps": exp["num_steps"],
        "batch_size": exp["batch_size"],
        "learning_rate": exp["learning_rate"],
        "weight_decay": exp["weight_decay"],
        "n_layer": exp["n_layer"],
        "n_embd": exp["n_embd"],
        "n_head": exp["n_head"],
        "n_train_docs": exp["n_train_docs"],
        "n_eval": exp["n_eval"],
    })

manifest_df = pd.DataFrame(manifest_rows)
manifest_df.to_csv(output_root / "manifest.csv", index=False)

suite_df = pd.concat(suite_rows, ignore_index=True)
suite_df.to_csv(output_root / "suite_summary.csv", index=False)
suite_mean_summary = suite_df.groupby(
    ["family", "train_family", "model_kind", "suite"]
).agg(
    seeds=("seed", "nunique"),
    exact_constrained_mean=("exact_constrained", "mean"),
    exact_constrained_std=("exact_constrained", "std"),
    teacher_token_accuracy_mean=("teacher_token_accuracy", "mean"),
    teacher_token_accuracy_std=("teacher_token_accuracy", "std"),
).reset_index()
suite_mean_summary.to_csv(output_root / "suite_mean_summary.csv", index=False)

activation_df = pd.concat(activation_rows, ignore_index=True)
activation_df.to_csv(output_root / "carryflag_activation_raw.csv", index=False)

activation_mean_summary = activation_df.groupby(["family", "suite"]).agg(
    seeds=("seed", "nunique"),
    n=("target", "size"),
    pred_hundreds_eq_c2_rate=("pred_hundreds_eq_c2", "mean"),
    pred_hundreds_eq_target_rate=("pred_hundreds_eq_target", "mean"),
    flag_minus_true_mean=("flag_minus_true", "mean"),
    flag_minus_true_std=("flag_minus_true", "std"),
    prob_flag_mean=("prob_flag", "mean"),
    prob_true_mean=("prob_true", "mean"),
    att_to_low2_mean=("att_to_low2", "mean"),
    att_to_high_mean=("att_to_high", "mean"),
    att_to_prev_output_mean=("att_to_prev_output", "mean"),
    hidden_norm_mean=("hidden_norm", "mean"),
).reset_index()
activation_mean_summary.to_csv(output_root / "carryflag_activation_mean_summary.csv", index=False)

activation_by_c2 = activation_df.groupby(["family", "suite", "c2"]).agg(
    n=("target", "size"),
    pred_hundreds_eq_c2_rate=("pred_hundreds_eq_c2", "mean"),
    pred_hundreds_eq_target_rate=("pred_hundreds_eq_target", "mean"),
    flag_minus_true_mean=("flag_minus_true", "mean"),
    att_to_low2_mean=("att_to_low2", "mean"),
    att_to_high_mean=("att_to_high", "mean"),
).reset_index()

activation_by_c2.to_csv(output_root / "carryflag_activation_by_c2.csv", index=False)

# Position-wise output summaries from saved predictions
prediction_rows = []
for exp in EXPERIMENTS:
    pred_path = output_root / exp["name"] / "predictions.csv"
    pred_df = pd.read_csv(pred_path)
    pred_df["family"] = exp["family"]
    pred_df["seed"] = exp["seed"]
    prediction_rows.append(pred_df)

pred_all = pd.concat(prediction_rows, ignore_index=True)

def _zfill_row_target(row):
    return str(int(row["y"])).zfill(int(row["width_out"]))

def _zfill_row_pred(row):
    return str(row["teacher_pred_chars"]).zfill(int(row["width_out"]))

pred_all["target_str"] = pred_all.apply(_zfill_row_target, axis=1)
pred_all["pred_str"] = pred_all.apply(_zfill_row_pred, axis=1)

def position_name(width_out: int, pos: int) -> str:
    if width_out == 4:
        return ["thousands", "hundreds", "tens", "ones"][pos]
    if width_out == 3:
        return ["hundreds", "tens", "ones"][pos]
    return f"pos{pos}"

pos_rows = []
for _, row in pred_all.iterrows():
    width_out = int(row["width_out"])
    for pos in range(width_out):
        pos_rows.append({
            "family": row["family"],
            "seed": row["seed"],
            "suite": row["suite"],
            "pos": pos,
            "pos_name": position_name(width_out, pos),
            "acc": int(row["pred_str"][pos] == row["target_str"][pos]),
        })

pos_df = pd.DataFrame(pos_rows)
positionwise_seed_summary = pos_df.groupby(["family", "seed", "suite", "pos_name"]).agg(
    acc=("acc", "mean"),
).reset_index()
positionwise_seed_summary.to_csv(output_root / "positionwise_accuracy_seed_summary.csv", index=False)

positionwise_mean_summary = positionwise_seed_summary.groupby(["family", "suite", "pos_name"]).agg(
    seeds=("seed", "nunique"),
    acc_mean=("acc", "mean"),
    acc_std=("acc", "std"),
).reset_index()
positionwise_mean_summary.to_csv(output_root / "positionwise_accuracy_mean_summary.csv", index=False)

cond_rows = []
for fam in sorted(pred_all["family"].unique()):
    fam_df = pred_all[pred_all["family"] == fam].copy()
    for suite_name in ["true3_noincarry", "true3_withincarry", "true3_thousands_carry"]:
        sub = fam_df[fam_df["suite"] == suite_name].copy()
        correct_thousands = sub["pred_str"].str[0] == sub["target_str"].str[0]
        correct_hundreds = sub["pred_str"].str[1] == sub["target_str"].str[1]
        correct_tens = sub["pred_str"].str[2] == sub["target_str"].str[2]
        correct_ones = sub["pred_str"].str[3] == sub["target_str"].str[3]
        upper_correct = correct_thousands & correct_hundreds
        cond_rows.append({
            "family": fam,
            "suite": suite_name,
            "n": int(len(sub)),
            "upper_correct_rate": float(upper_correct.mean()),
            "exact_rate": float(sub["exact_constrained"].mean()),
            "lower_both_correct_given_upper": float((correct_tens & correct_ones)[upper_correct].mean()) if upper_correct.any() else np.nan,
            "tens_correct_given_upper": float(correct_tens[upper_correct].mean()) if upper_correct.any() else np.nan,
            "ones_correct_given_upper": float(correct_ones[upper_correct].mean()) if upper_correct.any() else np.nan,
        })

conditional_summary = pd.DataFrame(cond_rows)
conditional_summary.to_csv(output_root / "upper_lower_conditional_summary.csv", index=False)

# Figures
plot_df = activation_mean_summary.copy()
suite_order = ["probe_h0", "probe_h1", "probe_t1"]
family_order = [f for f in ["mixedlayout_pcsig_base", "mixedlayout_pcsig_ctrl_dup", "mixedlayout_pcsig_probe_all"] if f in plot_df["family"].unique()]
x_positions = np.arange(len(suite_order))
bar_width = 0.24

fig, ax = plt.subplots(figsize=(7.8, 3.8))
for i, fam in enumerate(family_order):
    sub = plot_df[plot_df["family"] == fam].set_index("suite").reindex(suite_order)
    xs = x_positions + (i - (len(family_order) - 1) / 2) * bar_width
    ax.bar(xs, sub["flag_minus_true_mean"].values, width=bar_width, label=fam)
    ax.errorbar(xs, sub["flag_minus_true_mean"].values, yerr=sub["flag_minus_true_std"].fillna(0).values, fmt="none", capsize=3)
ax.axhline(0.0, linestyle="--", linewidth=1)
ax.set_xticks(x_positions)
ax.set_xticklabels(suite_order)
ax.set_ylabel("logit(flag digit) - logit(true hundreds)")
ax.set_title("Stage17 carry-flag margin at hundreds prediction step")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, 1.30), ncol=2, frameon=True)
fig.tight_layout()
fig.savefig(output_root / "carry_flag_margin.png", dpi=200, bbox_inches="tight")
plt.close(fig)

fig, ax = plt.subplots(figsize=(7.8, 3.8))
for i, fam in enumerate(family_order):
    sub = plot_df[plot_df["family"] == fam].set_index("suite").reindex(suite_order)
    xs = x_positions + (i - (len(family_order) - 1) / 2) * bar_width
    ax.bar(xs, sub["att_to_low2_mean"].values, width=bar_width, label=fam)
ax.set_xticks(x_positions)
ax.set_xticklabels(suite_order)
ax.set_ylabel("Mean final-layer attention mass to low2 operands")
ax.set_title("Stage17 attention to low-order operand digits at hundreds step")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, 1.30), ncol=2, frameon=True)
fig.tight_layout()

fig.savefig(output_root / "attention_to_lower_digits.png", dpi=200, bbox_inches="tight")
plt.close(fig)

# Position-wise accuracy figure
suite_order_pos = ["true3_noincarry", "true3_withincarry", "true3_thousands_carry"]
pos_order = ["thousands", "hundreds", "tens", "ones"]
x_map = {}
curr = 0.0
for suite_name in suite_order_pos:
    for j, pos_name in enumerate(pos_order):
        x_map[(suite_name, pos_name)] = curr + j
    curr += len(pos_order) + 0.8

fig, ax = plt.subplots(figsize=(10.5, 4.2))
for i, fam in enumerate(family_order):
    xs = []
    ys = []
    errs = []
    for suite_name in suite_order_pos:
        for pos_name in pos_order:
            sub = positionwise_mean_summary[
                (positionwise_mean_summary["family"] == fam)
                & (positionwise_mean_summary["suite"] == suite_name)
                & (positionwise_mean_summary["pos_name"] == pos_name)
            ]
            if len(sub) == 0:
                ys.append(np.nan)
                errs.append(0.0)
            else:
                ys.append(float(sub["acc_mean"].iloc[0]))
                errs.append(float(sub["acc_std"].iloc[0]) if not pd.isna(sub["acc_std"].iloc[0]) else 0.0)
            xs.append(x_map[(suite_name, pos_name)] + (i - (len(family_order) - 1) / 2) * bar_width)
    ax.bar(xs, ys, width=bar_width, label=fam)
    ax.errorbar(xs, ys, yerr=errs, fmt="none", capsize=2)

ax.set_ylim(0.0, 1.05)
ax.set_xticks([x_map[(suite_name, pos_name)] for suite_name in suite_order_pos for pos_name in pos_order])
ax.set_xticklabels(pos_order * len(suite_order_pos), rotation=45, ha="right")
for suite_name in suite_order_pos:
    center = np.mean([x_map[(suite_name, pos_name)] for pos_name in pos_order])
    ax.text(center, -0.17, suite_name, ha="center", va="top", transform=ax.get_xaxis_transform(), fontsize=10)
ax.set_ylabel("Position-wise teacher-forced accuracy")
ax.set_title("Stage17 position-wise output accuracy")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, 1.30), ncol=min(3, len(family_order)), frameon=True)
fig.tight_layout()
fig.savefig(output_root / "positionwise_accuracy.png", dpi=200, bbox_inches="tight")
plt.close(fig)

# Conditional residual lower-digit failure figure
cond_plot_order = ["true3_noincarry", "true3_withincarry", "true3_thousands_carry"]
x_positions = np.arange(len(cond_plot_order))
fig, ax = plt.subplots(figsize=(7.8, 3.8))
for i, fam in enumerate(family_order):
    ys = []
    for suite_name in cond_plot_order:
        sub = conditional_summary[
            (conditional_summary["family"] == fam)
            & (conditional_summary["suite"] == suite_name)
        ]
        ys.append(float(sub["lower_both_correct_given_upper"].iloc[0]) if len(sub) else np.nan)
    xs = x_positions + (i - (len(family_order) - 1) / 2) * bar_width
    ax.bar(xs, ys, width=bar_width, label=fam)
ax.set_xticks(x_positions)
ax.set_xticklabels(cond_plot_order)
ax.set_ylabel("P(tens & ones correct | thousands & hundreds correct)")
ax.set_title("Stage17 residual lower-digit failure after upper-digit repair")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, 1.30), ncol=min(3, len(family_order)), frameon=True)
fig.tight_layout()
fig.savefig(output_root / "lower_tail_error_given_upper_correct.png", dpi=200, bbox_inches="tight")
plt.close(fig)

readme = """\
Results pack contents (Stage17 carry-flag activation / attention probe):
- design_manifest.csv
- manifest.csv
- suite_summary.csv
- suite_mean_summary.csv
- carryflag_activation_raw.csv
- carryflag_activation_mean_summary.csv
- carryflag_activation_by_c2.csv
- carry_flag_margin.png
- attention_to_lower_digits.png
- positionwise_accuracy_mean_summary.csv
- positionwise_accuracy_seed_summary.csv
- upper_lower_conditional_summary.csv
- positionwise_accuracy.png
- lower_tail_error_given_upper_correct.png
- per-experiment folders with config / train_log / predictions / checkpoint / carryflag_activation_raw.csv

Interpretation:
- pred_hundreds_eq_c2_rate quantifies how often the hundreds prediction equals the incoming carry bit c2.
- flag_minus_true_mean > 0 means the carry-flag digit receives more logit support than the true hundreds digit.
- att_to_low2_mean measures mean final-layer attention mass from the hundreds-step query to a_tens/a_ones/b_tens/b_ones.

This notebook compares a base model, a matched extra-data control, and a targeted probe-all augmentation. It is designed as a mechanistic-support experiment for the carry-flag hypothesis, not a full circuit proof.
"""
(output_root / "README_results.txt").write_text(readme, encoding="utf-8")

zip_path = output_root.with_suffix(".zip")
if zip_path.exists():
    zip_path.unlink()
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    for path in output_root.rglob("*"):
        zf.write(path, path.as_posix())

print("done")
print("output_root:", output_root.resolve())
print("zip:", zip_path.resolve())
print(pd.read_csv(output_root / "carryflag_activation_mean_summary.csv"))

if AUTO_DOWNLOAD_ZIP:
    try:
        from google.colab import files
        files.download(str(zip_path))
    except Exception as e:
        print("auto-download skipped:", repr(e))
